### 1. INF

#### 在numpy中除以0并不会报错，会产生一个无穷大的数，即np.INF

In [1]:
import numpy as np
#创建一个数组并打印
a = np.random.randint(0,10,size=(3,4))
print(a)
a/0

[[1 6 7 8]
 [9 7 2 3]
 [1 4 6 9]]


c:\users\86188\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """


array([[inf, inf, inf, inf],
       [inf, inf, inf, inf],
       [inf, inf, inf, inf]])

### 2. NAN介绍

In [2]:
import numpy as np
#创建一个数组并打印
a = np.random.randint(0,10,size=(3,4))
print(a)

[[6 3 1 7]
 [8 8 9 2]
 [4 5 2 5]]


#### 将第一个数赋值为NAN

In [3]:
#将第一个数赋值为NAN
a[0,0]=np.NAN
#报错，整型和浮点类型无法进行赋值

ValueError: cannot convert float NaN to integer

#### 类型转换

In [4]:
#类型转换
a = a.astype(np.float)#回忆改变类型的方法
a[0,0] = np.NAN
print(a)

[[nan  3.  1.  7.]
 [ 8.  8.  9.  2.]
 [ 4.  5.  2.  5.]]


#### 特点1：NAN与任何数字运算的结果都是NAN

In [5]:
a/0

c:\users\86188\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[nan, inf, inf, inf],
       [inf, inf, inf, inf],
       [inf, inf, inf, inf]])

#### 特点2：NAN！=NAN

In [6]:
np.NAN == np.NAN

False

### 3. NAN的处理

In [7]:
#创建数组，赋值NAN
a = np.arange(12).reshape((3,4)).astype(np.float)
a[0,2],a[1,0] = np.NAN,np.NAN
a

array([[ 0.,  1., nan,  3.],
       [nan,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

#### 删除

In [8]:
#首先需要判断是不是NAN，不能使用==，提供了isnan方法
print(np.isnan(a))

[[False False  True False]
 [ True False False False]
 [False False False False]]


In [9]:
#如何提取呢？回忆上节课讲的布尔索引
a[np.isnan(a)]#提取到的是NAN的值

array([nan, nan])

In [10]:
#方法1：删除所有NAN值，其实就是提取所有非NAN的值
a[~np.isnan(a)]#提取到的是非NAN的值，扁平化为了一维数组

array([ 0.,  1.,  3.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [8]:
#方法2：删除NAN所在行，使用delete方法
#首先获取NAN所在行，where函数返回满足条件的元素所在的行和列
lines = np.where(np.isnan(a))[0]
print(lines)
a = np.delete(a,lines,axis=0)#delete函数参数的含义：数组本身；lines是一个列表，记录了行数；axis表示轴，0就是删除行，1就是删除列。
a

[0 1]


array([[ 8.,  9., 10., 11.]])

In [9]:
#关于where函数
a=np.array([[ 0.,  1., np.nan,  3.],
       [np.nan,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])
print(np.where(np.isnan(a)))

(array([0, 1], dtype=int64), array([2, 0], dtype=int64))


#### 替换NAN值

In [ ]:
#延后。
#从文件中读取数据，scores_nan.csv
scores = np.loadtxt('scores_nan.csv',delimiter=',',encoding='gb2312',skiprows=1,dtype=object)#读取为字符串，因为空值无法转换为float（默认值）
#将空字符串转换为nan
scores[scores==''] = 'nan'#当dtype为string时，字符串的长度是固定的，过长时会被截断，需要将dtype设置为object，所以不建议转化为nan
scores
#将所有转换为float
scores.astype(np.float)

In [10]:
#定义一个含有nan值的数组a
a=np.array([
    [ 0.,      1.,   np.nan],
    [np.nan,   5.,   6.],
    [ 8.,      9.,   10.],
    [6.,       2.,   6.]
])
print(a)

[[ 0.  1. nan]
 [nan  5.  6.]
 [ 8.  9. 10.]
 [ 6.  2.  6.]]


In [11]:
#将nan值替换为0
a[np.isnan(a)]=0
a

array([[ 0.,  1.,  0.],
       [ 0.,  5.,  6.],
       [ 8.,  9., 10.],
       [ 6.,  2.,  6.]])

##### 替换为均值
##### 首先熟悉如何做运算

In [ ]:
#求和运算
a.sum()#数组中所有元素的和

In [12]:
#按列求和，即求每一行的和
a.sum(axis=1)#！！0表示按行操作，1表示按列操作。这里axis=1，表示按列求和，即某一行的所有列相加，得出结果！！！

array([ 1., 11., 27., 14.])

In [14]:
#将nan赋值为每一列的平均值
#声明一个新数组
b = np.array([
    [ 0.,      1.,   np.nan],
    [np.nan,   5.,   6.],
    [ 8.,      9.,   10.],
    [6.,       2.,   6.]
])
b

array([[ 0.,  1., nan],
       [nan,  5.,  6.],
       [ 8.,  9., 10.],
       [ 6.,  2.,  6.]])

##### 步骤1：先获取列数，对每一列进行操作
##### 步骤2：求每一列，除nan外的均值
##### 步骤3：将均值赋值给nan

In [15]:
#使用shape属性获取列数
cols = b.shape[1]#shape返回的是行数和列数
#对每一列进行操作
for x in range(cols):
    #获取第x列
    col = b[:,x]
    #获取非nan值
    non_col = col[~np.isnan(col)]
    #求均值
    mean = non_col.mean()
    #赋值给nan
    col[np.isnan(col)] = mean
b

array([[ 0.        ,  1.        ,  7.33333333],
       [ 4.66666667,  5.        ,  6.        ],
       [ 8.        ,  9.        , 10.        ],
       [ 6.        ,  2.        ,  6.        ]])

In [ ]:
#在画图的时候，例如折线图，可能替换为均值更合理
#使用numpy处理文件，使用的频率不高，后期要讲的pandas使用频率更高，但是思路基本是一致的。